In [67]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset
import pandas as pd
from datetime import date, datetime, timedelta
import numpy as np
from azure.storage.blob import BlobServiceClient

subscription_id = '96599b9b-0d9b-4577-9163-418d2c6cd411'
resource_group = 'rg-datasci-ml-dev-001'
workspace_name = 'ml-moss-dev-001'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset_hourly_compare_cmic = Dataset.get_by_name(workspace, name='Hourly_Employee_Time_prod')
cmic_pr_df_dev = dataset_hourly_compare_cmic.to_pandas_dataframe()

dataset_hourly_compare_wd = Dataset.get_by_name(workspace, name='WD_Hourly_Employee_Time_prod')
wd_pr_df_dev = dataset_hourly_compare_wd.to_pandas_dataframe()

tcp_dataset = Dataset.get_by_name(workspace, name='WorkSegments_Timesheet_prod')
tcp_df_dev = tcp_dataset.to_pandas_dataframe()

jcjob_dataset = Dataset.get_by_name(workspace, name='JCJOB_TABLE_TN_prod')
jcjob_df = jcjob_dataset.to_pandas_dataframe()

Message: rslex failed, falling back to clex.
Payload: {"pid": 5434, "source": "azureml.dataprep", "version": "4.11.3", "trace": "azureml|data|tabular_dataset.py, line 169 in function <lambda>.\nazureml|data|dataset_error_handling.py, line 107 in function _try_execute.\nazureml|data|tabular_dataset.py, line 169 in function to_pandas_dataframe.", "subscription": "", "run_id": "", "resource_group": "", "workspace_name": "", "experiment_id": "", "location": "", "rslex_version": "2.18.3"}


In [68]:
##########################

week_ending = '2023-10-14'

##########################

date = datetime.strptime(week_ending, "%Y-%m-%d")
start_date = date - timedelta(days = 6)
end_date = date

In [69]:
tcp_df = tcp_df_dev.copy()

In [70]:
###TCP Data Manipulation
#Filter: Solar Jobs only 
tcp_df = tcp_df.loc[tcp_df['Job_Code'].str.startswith('710')]
#Filter: Only Keeping Labor Hourly
tcp_df = tcp_df.loc[tcp_df['Task'].isin(['LH','LHA'])]
#Filter: Only keeping Moss Labor Hourly - excluding subs
tcp_df = tcp_df.loc[tcp_df['Department']=='710']
#Creating a total hours column
tcp_df['total_hours'] = tcp_df['Regular_Hours'] + tcp_df['Overtime_Hours']
#Renaming Relevant Columns
tcp_df = tcp_df.rename({'Employee_Number':'EEID','Job_Code':'Job','Activity':'Sub_Job','total_hours':'Hours'},axis='columns')
tcp_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
tcp_df_comp = tcp_df.loc[:,tcp_cols_keep]

In [128]:
wd_pr_df = wd_pr_df_dev.copy()

In [123]:
#Workday Data Manipulation
#Drop Null values if the employee does not have a job/sub job assignment
wd_pr_df = wd_pr_df.loc[~wd_pr_df['billableProject.descriptor'].isna()]
#Filter: Solar Jobs Only
wd_pr_df = wd_pr_df.loc[wd_pr_df['billableProject.descriptor'].str.startswith('710')]
#Extracting Employee ID (EEID) from 'Workers' column: This column also designates employees as Terminated so the substring '(Termin)
wd_pr_df['Workers'] = wd_pr_df['Workers'].str.replace('(Terminated)' , '')
wd_pr_df['EEID'] = wd_pr_df['Workers'].str.split('(').str[1]
wd_pr_df['EEID'] = wd_pr_df['EEID'].str.split(')').str[0]
#This isolates the 2 digit sub job, also accomodates the sub jobs that have a letter in them like Claims (C) and Warranty (W)
wd_pr_df['billableProject.descriptor'] = wd_pr_df['billableProject.descriptor'].str.replace('C' , '.C')
wd_pr_df['billableProject.descriptor'] = wd_pr_df['billableProject.descriptor'].str.replace('W' , '.W')
wd_pr_df['sub_job'] = wd_pr_df['billableProject.descriptor'].str.split('.').str[1]
wd_pr_df['sub_job'] = wd_pr_df['sub_job'].str.split(' ').str[0]
#Leaves behind Job ONLY
wd_pr_df['job'] = wd_pr_df.loc[:,'billableProject.descriptor'].str[0:7]
#Renaming Relevant Columns
wd_pr_df = wd_pr_df.rename({'calculatedDate':'Date','job':'Job','sub_job':'Sub_Job','amount':'Hours'},axis='columns')
wd_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
wd_pr_df_comp = wd_pr_df.loc[:,wd_cols_keep]
#excluding these employees from Workday because they will not be present in the TCP data
wd_eeid_exclude = ['6423','6525','9633','9647','9686','11780','13401','13757','14452','14523','14587','14594','15312','15358','15681','15912']
wd_pr_df_comp = wd_pr_df_comp.loc[~wd_pr_df_comp['EEID'].isin(wd_eeid_exclude)]

In [18]:
cmic_pr_df = cmic_pr_df_dev.copy()

In [19]:
#CMiC Data Manipulation
#Re-casting this column as a Date data type
cmic_pr_df["Date"] = pd.to_datetime(cmic_pr_df["Date"])
#Filter: Only Keeping Labor Hourly
cmic_pr_df = cmic_pr_df.loc[cmic_pr_df['Task'].isin(['LH','LHA'])]
#Renaming Relevant Columns
cmic_pr_df = cmic_pr_df.rename({'Employee Number':'EEID','Job Code':'Job','Sub Job Code':'Sub_Job','Regular Hours':'Hours'},axis='columns')
cmic_cols_keep = ['EEID' , 'Date' , 'Job' , 'Sub_Job' , 'Hours']
cmic_pr_df_comp = cmic_pr_df.loc[:,cmic_cols_keep]

In [36]:
#Date Filters
tcp_filter = (tcp_df_comp['Date']>=start_date)&(tcp_df_comp['Date']<=end_date)
cmic_filter = (cmic_pr_df_comp['Date']>=start_date)&(cmic_pr_df_comp['Date']<=end_date)
wd_filter = (wd_pr_df_comp['Date']>=start_date)&(wd_pr_df_comp['Date']<=end_date)

#### Grouping the Data by Different Levels

In [183]:
#Grouping by Job
tcp_grouped_job = tcp_df_comp.loc[tcp_filter].groupby(['Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_job = tcp_grouped_job.reset_index()
cmic_grouped_job = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_job = cmic_grouped_job.reset_index()
wd_grouped_job = wd_pr_df_comp.loc[wd_filter].groupby(['Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_job = wd_grouped_job.reset_index()

In [184]:
#Grouping by Job, Sub Job
tcp_grouped_subjob = tcp_df_comp.loc[tcp_filter].groupby(['Job','Sub_Job']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_subjob = tcp_grouped_subjob.reset_index()
cmic_grouped_subjob = cmic_pr_df_comp.loc[cmic_filter].groupby(['Job','Sub_Job']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_subjob = cmic_grouped_subjob.reset_index()
wd_grouped_subjob = wd_pr_df_comp.loc[wd_filter].groupby(['Job','Sub_Job']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_subjob = wd_grouped_subjob.reset_index()

In [185]:
#Grouping by Job, Sub Job, EEID
tcp_grouped_ee = tcp_df_comp.loc[tcp_filter].groupby(['EEID']).agg(TotalHours_TCP = pd.NamedAgg(column='Hours',aggfunc='sum'))
tcp_grouped_ee = tcp_grouped_ee.reset_index()
cmic_grouped_ee = cmic_pr_df_comp.loc[cmic_filter].groupby(['EEID']).agg(TotalHours_CMiC = pd.NamedAgg(column='Hours',aggfunc='sum'))
cmic_grouped_ee = cmic_grouped_ee.reset_index()
wd_grouped_ee = wd_pr_df_comp.loc[wd_filter].groupby(['EEID']).agg(TotalHours_WD = pd.NamedAgg(column='Hours',aggfunc='sum'))
wd_grouped_ee = wd_grouped_ee.reset_index()

#### Unique Column Creation for the Joins

In [186]:
#Unique Columns to Join On for Sub Job Comparison
tcp_grouped_subjob['unique'] = tcp_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
cmic_grouped_subjob['unique'] = cmic_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)
wd_grouped_subjob['unique'] = wd_grouped_subjob[['Job','Sub_Job']].apply(lambda x: ' - '.join(str(value) for value in x), axis=1)

#### TCP vs. Workday Comparison

In [187]:
#TCP v. WD Job
tcp_v_wd_job = tcp_grouped_job.merge(wd_grouped_job , how = 'left' , on = 'Job' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_job['TotalHours_WD'] = tcp_v_wd_job['TotalHours_WD'].fillna(0)
tcp_v_wd_job['TotalHours_TCP'] = tcp_v_wd_job['TotalHours_TCP'].fillna(0)
tcp_v_wd_job['Hours_Variance (TCP - WD)'] = tcp_v_wd_job['TotalHours_TCP'] - tcp_v_wd_job['TotalHours_WD']
tcp_v_wd_job = tcp_v_wd_job.sort_values(['Job'  , 'Hours_Variance (TCP - WD)'] , ascending=[False,False])

In [188]:
#TCP v. WD Job, Sub Job
tcp_v_wd_subjob = tcp_grouped_subjob.merge(wd_grouped_subjob , how = 'left' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_wd_subjob['TotalHours_WD'] = tcp_v_wd_subjob['TotalHours_WD'].fillna(0)
tcp_v_wd_subjob['TotalHours_TCP'] = tcp_v_wd_subjob['TotalHours_TCP'].fillna(0)
tcp_v_wd_subjob['Hours_Variance (TCP - WD)'] = tcp_v_wd_subjob['TotalHours_TCP'] - tcp_v_wd_subjob['TotalHours_WD']
tcp_v_wd_subjob = tcp_v_wd_subjob.sort_values(['Job_TCP' , 'Sub_Job_TCP' , 'Hours_Variance (TCP - WD)'] , ascending=[False,True,False])

In [189]:
#TCP v. WD Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_TCP','_WD']).drop(columns =['unique'])
tcp_v_wd_ee = tcp_grouped_ee.merge(wd_grouped_ee , how = 'left' , on = 'EEID' , suffixes = ['_TCP','_WD'])
#NaN Hours will be filled with 0
tcp_v_wd_ee['TotalHours_WD'] = tcp_v_wd_ee['TotalHours_WD'].fillna(0)
tcp_v_wd_ee['TotalHours_TCP'] = tcp_v_wd_ee['TotalHours_TCP'].fillna(0)
tcp_v_wd_ee['Hours_Variance (TCP - WD)'] = tcp_v_wd_ee['TotalHours_TCP'] - tcp_v_wd_ee['TotalHours_WD']
tcp_v_wd_ee = tcp_v_wd_ee.sort_values('Hours_Variance (TCP - WD)' , ascending=False, key=abs)
#Removing Variances that are 0 - excluding EEIDs that match
tcp_v_wd_ee = tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0.0]

#### TCP vs. CMiC Comparison

In [190]:
#TCP v CMiC Job
tcp_v_cmic_job = tcp_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_job['TotalHours_CMiC'] = tcp_v_cmic_job['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_job['TotalHours_TCP'] = tcp_v_cmic_job['TotalHours_TCP'].fillna(0)
tcp_v_cmic_job['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_job['TotalHours_TCP'] - tcp_v_cmic_job['TotalHours_CMiC']
tcp_v_cmic_job = tcp_v_cmic_job.sort_values(['Job' , 'Hours_Variance (TCP - CMiC)'], ascending=[False, False])

In [191]:
#TCP v CMiC Job, Sub Job
tcp_v_cmic_subjob = tcp_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
tcp_v_cmic_subjob['TotalHours_CMiC'] = tcp_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_subjob['TotalHours_TCP'] = tcp_v_cmic_subjob['TotalHours_TCP'].fillna(0)
tcp_v_cmic_subjob['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_subjob['TotalHours_TCP'] - tcp_v_cmic_subjob['TotalHours_CMiC']
tcp_v_cmic_subjob = tcp_v_cmic_subjob.sort_values(['Job' , 'Sub_Job', 'Hours_Variance (TCP - CMiC)'], ascending=[False, True, False])

In [192]:
#TCP v CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['','_CMiC']).drop(columns =['unique'])
tcp_v_cmic_ee = tcp_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['','_CMiC'])
#NaN Hours will be filled with 0
tcp_v_cmic_ee['TotalHours_CMiC'] = tcp_v_cmic_ee['TotalHours_CMiC'].fillna(0)
tcp_v_cmic_ee['TotalHours_TCP'] = tcp_v_cmic_ee['TotalHours_TCP'].fillna(0)
tcp_v_cmic_ee['Hours_Variance (TCP - CMiC)'] = tcp_v_cmic_ee['TotalHours_TCP'] - tcp_v_cmic_ee['TotalHours_CMiC']
tcp_v_cmic_ee = tcp_v_cmic_ee.sort_values('Hours_Variance (TCP - CMiC)', ascending=False, key=abs)
#Filter down to relevant columns only

#### Workday vs. CMiC Comparison

In [193]:
#WD v. CMiC EEID
wd_v_cmic_job = wd_grouped_job.merge(cmic_grouped_job , how = 'outer' , on = 'Job' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_job['TotalHours_CMiC'] = wd_v_cmic_job['TotalHours_CMiC'].fillna(0)
wd_v_cmic_job['TotalHours_WD'] = wd_v_cmic_job['TotalHours_WD'].fillna(0)
wd_v_cmic_job['Hours_Variance (WD - CMiC)'] = wd_v_cmic_job['TotalHours_WD'] - wd_v_cmic_job['TotalHours_CMiC']
wd_v_cmic_job = wd_v_cmic_job.sort_values(['Job' , 'Hours_Variance (WD - CMiC)'], ascending=[False,False])

In [194]:
#WD v. CMiC Job, Sub Job
wd_v_cmic_subjob = wd_grouped_subjob.merge(cmic_grouped_subjob , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
#NaN Hours will be filled with 0
wd_v_cmic_subjob['TotalHours_CMiC'] = wd_v_cmic_subjob['TotalHours_CMiC'].fillna(0)
wd_v_cmic_subjob['TotalHours_WD'] = wd_v_cmic_subjob['TotalHours_WD'].fillna(0)
wd_v_cmic_subjob['Hours_Variance (WD - CMiC)'] = wd_v_cmic_subjob['TotalHours_WD'] - wd_v_cmic_subjob['TotalHours_CMiC']
wd_v_cmic_subjob = wd_v_cmic_subjob.sort_values(['Job_WD' , 'Sub_Job_WD' , 'Hours_Variance (WD - CMiC)'], ascending=[False,True,False])

In [195]:
#WD v. CMiC Job, Sub Job, EEID
### Commented out for EEID Comparison Only - uncomment out to compare EEID by Job and Sub Job
#wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'unique' , suffixes = ['_WD','_CMiC']).drop(columns =['unique'])
wd_v_cmic_ee = wd_grouped_ee.merge(cmic_grouped_ee , how = 'outer' , on = 'EEID' , suffixes = ['_WD','_CMiC'])
#NaN Hours will be filled with 0
wd_v_cmic_ee['TotalHours_CMiC'] = wd_v_cmic_ee['TotalHours_CMiC'].fillna(0)
wd_v_cmic_ee['TotalHours_WD'] = wd_v_cmic_ee['TotalHours_WD'].fillna(0)
wd_v_cmic_ee['Hours_Variance (WD - CMiC)'] = wd_v_cmic_ee['TotalHours_WD'] - wd_v_cmic_ee['TotalHours_CMiC']
wd_v_cmic_ee = wd_v_cmic_ee.sort_values('Hours_Variance (WD - CMiC)', ascending=False , key=abs)
#Filter down to relevant columns only

In [196]:
file_path = os.getcwd() + '/excel_output'
file_path

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/jbeachy2/code/Users/JBeachy/Project_DataDomain/Payroll/git_LaborHours_Compare/excel_output'

In [197]:
#File Path in AMLS
file_name = f'Hours_Comparison_WeekEnding_{week_ending}'
#Writing to AMLS File Directory
with pd.ExcelWriter(f'{file_path}/{file_name}.xlsx') as writer:
    tcp_v_wd_job.to_excel(writer , sheet_name = 'TCPvWD_Hours_byJob' , index = False)
    tcp_v_wd_subjob.to_excel(writer , sheet_name = 'TCPvWD_Hours_bySubJob' , index = False)
    tcp_v_wd_ee.to_excel(writer , sheet_name = 'TCPvWD_Hours_byEE' , index = False)
    tcp_v_cmic_job.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byJob' , index = False)
    tcp_v_cmic_subjob.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_bySubJob' , index = False)
    tcp_v_cmic_ee.to_excel(writer , sheet_name = 'TCPvCMiC_Hours_byEE' , index = False)
    wd_v_cmic_job.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byJob' , index = False)
    wd_v_cmic_subjob.to_excel(writer , sheet_name = 'WDvCMiC_Hours_bySubJob' , index = False)
    wd_v_cmic_ee.to_excel(writer , sheet_name = 'WDvCMiC_Hours_byEE' , index = False)

In [198]:
#Blob Storage Info
storage_account_key = 'u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA=='
storage_account_name = 'mossdatalakesource'
connection_string = 'DefaultEndpointsProtocol=https;AccountName=mossdatalakesource;AccountKey=u46QXBaayH/rWljcqPTWZTNgFdGHo9zH4I0OLWgDi4oa2inkpjVwTbp74C+ISDC2oWtNlrcr69Ec+ASt5wV1PA==;EndpointSuffix=core.windows.net'
container_name = 'cmic'
#Defininng Blob Storage Uplpoad#
def uploadtoblobstorage(file_path,file_name):
    blob_service_client = BlobServiceClient.from_connection_string(connection_string)
    blob_client = blob_service_client.get_blob_client(container = container_name , blob = file_name)

    with open(file_path , 'rb') as data:
        blob_client.upload_blob(data)
    print(f'Uploaded {file_name}.')

In [199]:
file_name_exp = f'{file_name}.xlsx'
uploadtoblobstorage(f'{file_path}/{file_name}.xlsx', file_name_exp)

Uploaded Hours_Comparison_WeekEnding_2023-10-14.xlsx.


#### Run Statistics 

In [200]:
#### TCP vs. Workday Stats
today = date.today()
today_timechange = today - timedelta(hours=4)
today_for_text = today_timechange.strftime("%m-%d-%Y %H:%M:%S")
print("TCP vs. Workday")
print("Count of EEIDs that are showing Zero Hours Worked in Workday: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_WD'] == 0]))
print("Count of EEIDs that are showing Zero Hours Worked in TCP: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['TotalHours_TCP'] == 0]))
print("Count of EEIDs with a variance less than 10 hours(ABS), not including 0: ", len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']< 10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] >-10)&(tcp_v_wd_ee['Hours_Variance (TCP - WD)'] != 0)]))
print("Count of EEIDs with no variance in Hours Worked: " , len(tcp_v_wd_ee.loc[tcp_v_wd_ee['Hours_Variance (TCP - WD)'] == 0]))
print("Count of EEIDs with variance greater than or equal to 10(ABS): ",len(tcp_v_wd_ee.loc[(tcp_v_wd_ee['Hours_Variance (TCP - WD)']>=10)|(tcp_v_wd_ee['Hours_Variance (TCP - WD)']<=-10)]))
print(f"Ran on {today_for_text} for Week Ending {week_ending}")

TCP vs. Workday
Count of EEIDs that are showing Zero Hours Worked in Workday:  76
Count of EEIDs that are showing Zero Hours Worked in TCP:  0
Count of EEIDs with a variance less than 10 hours(ABS), not including 0:  5
Count of EEIDs with no variance in Hours Worked:  0
Count of EEIDs with variance greater than or equal to 10(ABS):  72
Ran on 10-17-2023 12:50:08 for Week Ending 2023-10-14


In [209]:
tcp_df_dev.loc[(tcp_df_dev['Employee_Number']=='20763')&tcp_filter].sort_values('Date',ascending = True)

,WorkSegments_Timesheet_ID,Employee_Number,First_Name,Last_Name,Date,Job_Code,Activity,Phase_Code,Task,Regular_Hours,Overtime_Hours,Pay_Rate,Employee_Classification,Department,FileName,Created_Date


In [106]:
#Date Filters
tcp_filter = (tcp_df_comp['Date']>=start_date)&(tcp_df_comp['Date']<=end_date)
cmic_filter = (cmic_pr_df_comp['Date']>=start_date)&(cmic_pr_df_comp['Date']<=end_date)
wd_filter = (wd_pr_df_comp['Date']>=start_date)&(wd_pr_df_comp['Date']<=end_date)

In [107]:
#['EEID','Date','Job','Sub_Job','Hours']
wd_eeid = list(wd_pr_df_comp.loc[wd_filter,'EEID'].unique())
tcp_eeid = list(tcp_df.loc[tcp_filter,'EEID'].unique())

In [108]:
eeid_intcp_not_wd = list(set(tcp_eeid) - set(wd_eeid))
len(eeid_intcp_not_wd)

34

In [109]:
for x in eeid_intcp_not_wd:
    print(x)

6937
16200
14705
5792
15564
15033
15962
5243000043
14987
7811
16187
11143
5962
11089
15990
15454
16138
16324
13279
15762
16030
16165
16180
12968
15369
14829
5105
20500
13732
20619
5243000015
5869
16162
20306


In [86]:
wd_eeid_explore = wd_pr_df_dev.copy()

In [87]:
wd_eeid_explore['EEID'] = wd_eeid_explore['Workers'].str.split('(').str[1]
wd_eeid_explore['EEID'] = wd_eeid_explore['EEID'].str.split(')').str[0]

In [105]:
wd_pr_df_comp.loc[wd_pr_df_comp['EEID'] == '4082']

,EEID,Date,Job,Sub_Job,Hours
43021,4082,2023-10-02,7102023,15,10.0
43022,4082,2023-10-03,7102023,15,10.0
43023,4082,2023-10-04,7102023,15,10.0
43024,4082,2023-10-05,7102023,15,10.0
50602,4082,2023-10-09,7102023,20,10.0
50603,4082,2023-10-10,7102023,15,10.0
50604,4082,2023-10-11,7102023,20,10.0
50605,4082,2023-10-12,7102023,15,10.0
51896,4082,2023-10-09,7102023,20,10.0
51897,4082,2023-10-10,7102023,15,10.0


In [116]:
#payrollStatusForCalculatedTimeBlock
wd_pr_df_dev#['billableProject.descriptor'].unique()

,workdayID,EmployeeID,Employee_FullName,Workers,Employee_BillTransaction_Descriptor,billableTransaction.descriptor,Employee_BillTransactionDate,amount,dayOfTheWeek,FrequencySalary,...,timeType1.id,totalHoursTimeTracking,transactionDate2,transactionSource.descriptor,transactionSource.id,unit.descriptor,unit.id,unit1,unitOfTimeIsHours,unroundedDuration
0,aa485b7fc50c9001f415ba095fa80000,3619,Pamela Arteaga,Pamela Arteaga (3619),0.5,0.5 on 10/06/2023,06/2023,0.50,Friday,Bi-Weekly Salary,...,88572bc8f2d81000d33178c95c170001,0.50,2023-10-06,Time,7a55a775b63b4b9b8a5b3722fc1e4556,None,None,None,False,0.00
1,07f132c6a9069001f2beb40701530000,5178,Frank Young,Frank Young (5178),8 Hours,8 Hours on 10/17/2023,10/17/2023,8.00,Tuesday,Bi-Weekly Salary,...,88572bc8f2d81000d2ff785656e20000,8.00,2023-10-17,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.00
2,07f132c6a9069001f2c135643ccd0000,5178,Frank Young,Frank Young (5178),8 Hours,8 Hours on 10/18/2023,10/18/2023,8.00,Wednesday,Bi-Weekly Salary,...,88572bc8f2d81000d2ff785656e20000,8.00,2023-10-18,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.00
3,07f132c6a9069001f2c387709ccf0000,5178,Frank Young,Frank Young (5178),8 Hours,8 Hours on 10/19/2023,10/19/2023,8.00,Thursday,Bi-Weekly Salary,...,88572bc8f2d81000d2ff785656e20000,8.00,2023-10-19,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.00
4,07f132c6a9069001f2c69db2ab220000,5178,Frank Young,Frank Young (5178),8 Hours,8 Hours on 10/20/2023,10/20/2023,8.00,Friday,Bi-Weekly Salary,...,88572bc8f2d81000d2ff785656e20000,8.00,2023-10-20,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76557,07f132c6a9069000cd4715fc58590001,6585,Wayne Stevenson,Wayne Stevenson (6585),5.5 Hours,5.5 Hours on 09/27/2023,n 09/27/2023,5.50,Wednesday,Weekly Hourly,...,4a99fa1f113e100129b621416cf90001,5.50,2023-09-27,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,5.50
76558,07f132c6a9069000cd471695eeb60000,6585,Wayne Stevenson,Wayne Stevenson (6585),5.75 Hours,5.75 Hours on 09/26/2023,on 09/26/2023,5.75,Tuesday,Weekly Hourly,...,4a99fa1f113e100129b621416cf90001,5.75,2023-09-26,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,5.75
76559,07f132c6a9069000cd471695eeb60001,6585,Wayne Stevenson,Wayne Stevenson (6585),5.5 Hours,5.5 Hours on 09/28/2023,n 09/28/2023,5.50,Thursday,Weekly Hourly,...,4a99fa1f113e100129b621416cf90001,5.50,2023-09-28,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,5.50
76560,07f132c6a9069000cdc0b0922eca0000,14718,Keandra Adams,Keandra Adams (14718),2 Hours,2 Hours on 09/29/2023,09/29/2023,2.00,Friday,Bi-Weekly Salary,...,88572bc8f2d81000d2d23d22c2a00001,2.00,2023-09-29,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,2.00


In [130]:
wd_pr_df.loc[wd_pr_df['Employee_FullName'].str.contains('Kenneth Bowles')]

,workdayID,EmployeeID,Employee_FullName,Workers,Employee_BillTransaction_Descriptor,billableTransaction.descriptor,Employee_BillTransactionDate,amount,dayOfTheWeek,FrequencySalary,...,timeType1.id,totalHoursTimeTracking,transactionDate2,transactionSource.descriptor,transactionSource.id,unit.descriptor,unit.id,unit1,unitOfTimeIsHours,unroundedDuration
19606,1abc577aeaea9000c45ba00374ad0000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),5 Hours,5 Hours on 10/10/2023,10/10/2023,5.00,Tuesday,None,...,88572bc8f2d81000d2fc0169f9600000,5.00,2023-10-10,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
45056,babe2cdd23cd9000b541f31099f60000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),7.25 Hours,7.25 Hours on 09/26/2023,on 09/26/2023,7.25,Tuesday,None,...,88572bc8f2d81000d2fc0169f9600000,7.25,2023-09-26,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
45057,babe2cdd23cd9000b541f3aac2800000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),10 Hours,10 Hours on 09/27/2023,09/27/2023,10.00,Wednesday,None,...,88572bc8f2d81000d2fc0169f9600000,10.00,2023-09-27,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
45058,babe2cdd23cd9000b541f44487da0000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),10 Hours,10 Hours on 09/28/2023,09/28/2023,10.00,Thursday,None,...,88572bc8f2d81000d2fc0169f9600000,10.00,2023-09-28,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
65255,7c8f1553b8899000be0b3bcb9b9d0000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),10 Hours,10 Hours on 10/03/2023,10/03/2023,10.00,Tuesday,None,...,88572bc8f2d81000d2fc0169f9600000,10.00,2023-10-03,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
65256,7c8f1553b8899000be0b3c65d4410000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),10 Hours,10 Hours on 10/04/2023,10/04/2023,10.00,Wednesday,None,...,88572bc8f2d81000d2fc0169f9600000,10.00,2023-10-04,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
65257,7c8f1553b8899000be0b3cffca2f0000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),10.5 Hours,10.5 Hours on 10/05/2023,on 10/05/2023,10.50,Thursday,None,...,88572bc8f2d81000d2fc0169f9600000,10.50,2023-10-05,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
65258,7c8f1553b8899000be0b3d99bea80000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),9.5 Hours,9.5 Hours on 10/06/2023,n 10/06/2023,7.50,Friday,None,...,88572bc8f2d81000d2fc0169f9600000,7.50,2023-10-06,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
65259,7c8f1553b8899000be0b3e33fc950000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),2 Hours,2 Hours on 10/06/2023,10/06/2023,2.00,Friday,None,...,88572bc8f2d81000d2fc0169f9600000,2.00,2023-10-06,Time,7a55a775b63b4b9b8a5b3722fc1e4556,Hours,c4dacbd56bca4a9a8950e8d3ed21bbdb,Hours,True,0.0
65401,dcd88da845b59000bdf2ae109ec20000,15033,Kenneth Bowles,Kenneth Bowles (Terminated) (15033),2,2 on 10/06/2023,/2023,2.00,Friday,None,...,88572bc8f2d81000d2fc0169f9600000,2.00,2023-10-06,Time,7a55a775b63b4b9b8a5b3722fc1e4556,None,None,None,False,0.0


In [127]:
#.replace('(Terminated)' , '')

wd_pr_df.loc[wd_pr_df['EEID']=='Terminated','EmployeeID'].unique()

array(['14987', '16322', '20351', '20343', '14829', '16162', '16165',
       '16187', '16200', '16138', '-', '15505', '15962', '9145', '16120',
       '16171', '16172', '16174', '8994', '16255', '15762', '16318',
       '16316', '16249', '6102', '16312', '15855', '16305', '16304',
       '16302', '16298', '13408', '14707', '5733', '16019', '6024',
       '16023', '15612', '15705', '15954', '15613', '15800', '6868',
       '15086', '15176', '14855', '15469', '15458', '15468', '15456',
       '5105', '16193', '15034', '14653', '7629', '7864', '15792',
       '12899', '12968', '13609', '15983', '16074', '16329', '16346',
       '15454', '16363', '15949', '13732', '15590', '15640', '11128',
       '12897', '15255', '16357', '11632', '7546', '14705', '20406',
       '15742', '7249', '7417', '7435', '14711', '15854', '15851',
       '14112', '14221', '14236', '14343', '14514', '14905', '14921',
       '14952', '14972', '15523', '15536', '5365', '20316', '13279',
       '13716', '20547', '123

In [129]:
wd_pr_df = wd_pr_df_dev.copy()

In [139]:
wd_pr_df['Workers'] = wd_pr_df['Workers'].str.replace('(Terminated)' , '')
wd_pr_df['EEID'] = wd_pr_df['Workers'].str.split('(').str[1]
wd_pr_df['EEID'] = wd_pr_df['EEID'].str.split(')').str[0]

In [141]:
#also check EEID 15033
wd_pr_df.loc[wd_pr_df['Employee_FullName'].str.contains('Kenneth Bowles'),'EEID']

19606    15033
45056    15033
45057    15033
45058    15033
65255    15033
65256    15033
65257    15033
65258    15033
65259    15033
65401    15033
Name: EEID, dtype: object